<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/27_ntb_PySpark_ReadFunctions_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Tecnologias Utilizadas:
- Pyspark
- Python

## Contato:
Se precisar de mais informações, entre em contato:

**Email: luciana.sampaio84@gmail.com**

LinkedIn: [Luciana Sampaio ](https://www.linkedin.com/in/luciana-sampaio/)


In [5]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
import io
import warnings
warnings.filterwarnings("ignore")


In [6]:
# Cria uma SparkSession
spark = SparkSession.builder.appName("diabetes").getOrCreate()

# Link direto para download do arquivo
url = 'https://github.com/luasampaio/ModeloPredi-oDiabetes/blob/main/diabetes.csv'

In [8]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
import io
import warnings
warnings.filterwarnings("ignore")

# Cria uma SparkSession
spark = SparkSession.builder.appName("diabetes").getOrCreate()

# Link direto para download do arquivo - Changed to raw content URL
url = 'https://github.com/luasampaio/ModeloPredi-oDiabetes/raw/main/diabetes.csv'

# Download the CSV file using requests
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad responses

# Read the CSV data into a Pandas DataFrame
diabetes = pd.read_csv(io.StringIO(response.text), sep=',')


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [15]:
# Convert the Pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(diabetes)

In [18]:

df.show(4)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 4 rows



In [19]:
df.printSchema()

root
 |-- Pregnancies: long (nullable = true)
 |-- Glucose: long (nullable = true)
 |-- BloodPressure: long (nullable = true)
 |-- SkinThickness: long (nullable = true)
 |-- Insulin: long (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: long (nullable = true)
 |-- Outcome: long (nullable = true)



In [20]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [22]:
# Columns of dataframe
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [23]:
# Shape of dataset
print((df.count(),len(df.columns)))

(768, 9)


# Distribuição Estatística


In [24]:
# description statistics
df.describe().show(truncate=8)

+-------+-----------+--------+-------------+-------------+--------+--------+------------------------+--------+--------+
|summary|Pregnancies| Glucose|BloodPressure|SkinThickness| Insulin|     BMI|DiabetesPedigreeFunction|     Age| Outcome|
+-------+-----------+--------+-------------+-------------+--------+--------+------------------------+--------+--------+
|  count|        768|     768|          768|          768|     768|     768|                     768|     768|     768|
|   mean|   3.845...|120.8...|     69.10...|     20.53...|79.79...|31.99...|                0.471...|33.24...|0.348...|
| stddev|   3.369...|31.97...|     19.35...|     15.95...|115.2...|7.884...|                0.331...|11.76...|0.476...|
|    min|          0|       0|            0|            0|       0|     0.0|                   0.078|      21|       0|
|    max|         17|     199|          122|           99|     846|    67.1|                    2.42|      81|       1|
+-------+-----------+--------+----------

# Filtrando

In [25]:
# Selecionando somente duas colunas
df.select("Pregnancies", "Age").show(10)

+-----------+---+
|Pregnancies|Age|
+-----------+---+
|          6| 50|
|          1| 31|
|          8| 32|
|          1| 21|
|          0| 33|
|          5| 30|
|          3| 26|
|         10| 29|
|          2| 53|
|          8| 54|
+-----------+---+
only showing top 10 rows



In [26]:
from pyspark.sql.functions import current_date


# Add current date column
df = df.withColumn("date", current_date())


In [27]:
df.show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|      date|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|2025-01-27|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|2025-01-27|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|2025-01-27|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|2025-01-27|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|2025-01-27|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|2025-01-27|
|

In [28]:
# Segunda opção de usar o Metodo Col
import pyspark.sql.functions as F
df.select(F.col("Pregnancies"), F.col("Age")).show(10)

+-----------+---+
|Pregnancies|Age|
+-----------+---+
|          6| 50|
|          1| 31|
|          8| 32|
|          1| 21|
|          0| 33|
|          5| 30|
|          3| 26|
|         10| 29|
|          2| 53|
|          8| 54|
+-----------+---+
only showing top 10 rows



#Filtrando somente os pacientes menores de  40 anos

In [29]:
# Filtering
df.filter(df['Age'] < 40).show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|      date|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|2025-01-27|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|2025-01-27|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|2025-01-27|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|2025-01-27|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|2025-01-27|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|2025-01-27|
|

In [30]:
# Filtrando duas Colunas

df.filter(df['age'] < 40).select('Insulin','Outcome').show(10)

+-------+-------+
|Insulin|Outcome|
+-------+-------+
|      0|      0|
|      0|      1|
|     94|      0|
|    168|      1|
|      0|      0|
|     88|      1|
|      0|      0|
|      0|      0|
|      0|      1|
|      0|      1|
+-------+-------+
only showing top 10 rows



In [ ]:
# Filtrando com duas colunas e com uma condicional
df.filter(df['age'] > 60).filter(df['Outcome'] == '1').show(10)

In [31]:
# Filtrando com duas condições
df.filter((df['Outcome']==1) & (df['Pregnancies'] >=9)).show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|      date|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+
|         10|    168|           74|            0|      0|38.0|                   0.537| 34|      1|2025-01-27|
|          9|    119|           80|           35|      0|29.0|                   0.263| 29|      1|2025-01-27|
|         11|    143|           94|           33|    146|36.6|                   0.254| 51|      1|2025-01-27|
|         10|    125|           70|           26|    115|31.1|                   0.205| 41|      1|2025-01-27|
|          9|    102|           76|           37|      0|32.9|                   0.665| 46|      1|2025-01-27|
|          9|    171|          110|           24|    240|45.4|                   0.721| 54|      1|2025-01-27|
|

In [ ]:
# Filtrando usando o metodo where e usando o count
df.where(df['age']>40).count()

# Groupby

In [32]:
# Grouping data
df.groupBy('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      0|  500|
|      1|  268|
+-------+-----+



## Valores Distintos

In [ ]:
# valores distintos
df.select('Pregnancies').distinct().count()

In [33]:
#  Idade Media com e sem diabetes
df.groupBy('Outcome').agg(F.mean("age").alias("age_mean")).show(10)

+-------+-----------------+
|Outcome|         age_mean|
+-------+-----------------+
|      0|            31.19|
|      1|37.06716417910448|
+-------+-----------------+



In [35]:

from pyspark.sql import functions as F

# Calculate the mean age grouped by Outcome
result = df.groupBy('Outcome').agg(F.mean("age").alias("age_mean"))
result.show(10)


for row in result.collect():
    print(f"Outcome: {row['Outcome']}, Idade média: {row['age_mean']:.2f}")


+-------+-----------------+
|Outcome|         age_mean|
+-------+-----------------+
|      0|            31.19|
|      1|37.06716417910448|
+-------+-----------------+

Outcome: 0, Idade média: 31.19
Outcome: 1, Idade média: 37.07


In [36]:
# Criando Função
def diabete(case):
    if case == 1 :
        return "diabete"
    else:
        return 'no diabete'

In [37]:
diabete_udf = F.udf(diabete)

In [38]:
# Aplicando a função
from pyspark.sql.types import *
diabete_udf = F.udf(diabete, StringType())
df.withColumn('diabete_case', diabete_udf(df['Outcome'])).show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|      date|diabete_case|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+----------+------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|2025-01-27|     diabete|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|2025-01-27|  no diabete|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|2025-01-27|     diabete|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|2025-01-27|  no diabete|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|2025-01-27|     diabete|
|       

In [39]:
# salvar em um csv
df.write.format("csv").option("header", "true").save("diabetes_csv")

In [41]:
# Download do arquivo CSV para o seu ambiente local
from google.colab import files
import os

# Define o nome do arquivo e diretório
output_filename = "diabetes.csv"
output_dir = "diabetes_csv"

# Verifica se existe algum arquivo dentro do diretório
file_list = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]

if file_list:
    # Seleciona o primeiro arquivo encontrado (assumindo que apenas um é gerado)
    output_file = os.path.join(output_dir, file_list[0])

    try:
        files.download(output_file)
        print(f"Arquivo '{output_file}' baixado com sucesso.")
    except Exception as e:
        print(f"Erro ao baixar o arquivo: {e}")
else:
    print("Nenhum arquivo encontrado no diretório especificado.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo 'diabetes_csv/._SUCCESS.crc' baixado com sucesso.
